In [8]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from  sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss,roc_auc_score
import re
from tqdm import tqdm

le = LabelEncoder()

In [14]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

X = pd.concat([train.drop(['click'],axis=1),test])
print(X.shape)


train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 34)


# fea 1. Get letter prefix -- not good to use

In [15]:
def get_prefix_letter(x):
    return ','.join(re.findall('(?:\w+|\d+)_',x))
            
tag_fe = pd.DataFrame()
nan_index = X[X['user_tags'].isnull()].index
tag_fe['prefix_letter'] = X['user_tags'].fillna('nan').apply(get_prefix_letter)

all_list = []
for each in tag_fe['prefix_letter'].values:
    all_list.extend(each.split(','))
unique_tag = list(set(all_list)-set(['']))

def extract_unique_tag(x,tag):
    finds = re.findall(tag,x)
    if len(finds) > 0:
        return 1
    else:
        return 0
# for each_tag in tqdm(unique_tag):
#     tag_fe[each_tag] = tag_fe['prefix_letter'].apply(extract_unique_tag,args=(each_tag,))
#     tag_fe.loc[nan_index,each_tag] = np.nan
#     tag_fe.rename({each_tag:'user_tag_'+each_tag},axis=1,inplace=True)
    

tag_fe['instance_id'] = X['instance_id'].values
tag_fe['prefix_letter'] = le.fit_transform(tag_fe['prefix_letter'])
tag_fe['prefix_letter'] = tag_fe['prefix_letter'].astype('category')
tag_fe.loc[nan_index,'prefix_letter'] = np.nan

# tag_fe.drop('prefix_letter',axis=1,inplace=True)

# fea 2 user_tag count

In [4]:
tag_fe = pd.DataFrame()
nan_index = X[X['user_tags'].isnull()].index
tag_fe['tags_count'] = X['user_tags'].fillna('notfound').apply(lambda x:len(x.split(',')))
tag_fe.loc[nan_index,'tags_count'] = 0
tag_fe['instance_id'] = X['instance_id'].values

In [16]:
train_base = pd.read_pickle(FILE.train_agg_v3_select.value)
print(train_base.shape)
test_base = pd.read_pickle(FILE.test_agg_v3_select.value)
print(test_base.shape)

(1001650, 35)
(40024, 34)


In [17]:
train_save = train_base.merge(tag_fe,how='inner',on='instance_id')
print(train_save.shape)
test_save = test_base.merge(tag_fe,how='inner',on='instance_id')
print(test_save.shape)

(1001650, 36)
(40024, 35)


In [18]:
train_save.to_pickle(FILE.train_appends.value)
test_save.to_pickle(FILE.test_appends.value)

In [68]:
tag_fe

,prefix_letter,user_tag_mz_ag_,user_tag_mzgd_,user_tag_gd_,user_tag_mzag_,user_tag_ag_,user_tag_mz_gd_,instance_id
0,,NaN,NaN,NaN,NaN,NaN,NaN,86294719979897807
1,,0.0,0.0,0.0,0.0,0.0,0.0,2699289844928136052
2,,NaN,NaN,NaN,NaN,NaN,NaN,3117527168445845752
3,gd_,NaN,NaN,NaN,NaN,NaN,NaN,3398484891050993371
4,,NaN,NaN,NaN,NaN,NaN,NaN,2035477570591176488
5,,0.0,0.0,0.0,0.0,0.0,0.0,2065527640347419040
6,,NaN,NaN,NaN,NaN,NaN,NaN,9137625226649553828
7,"gd_,ag_,gd_",NaN,NaN,NaN,NaN,NaN,NaN,8256661937587962948
8,,0.0,0.0,0.0,0.0,0.0,0.0,2270018983665706006
9,"ag_,gd_",0.0,0.0,1.0,0.0,1.0,0.0,5028130673845168373


In [3]:
def tags_sort(x):
    try:
        return ' '.join(sorted(list(set(x.split(',')) -set(['']))))
    except:
        return x
X['user_tags'] = X['user_tags'].apply(tags_sort)
X['user_tags'] = X['user_tags'].fillna('NotFound')

# Count Vectorize

In [4]:
ct = CountVectorizer(token_pattern=r'(?:\d+|\w+)',
                max_features=500000)
cv_tag = ct.fit_transform(X['user_tags'])

# TFIDF Vectorize

In [5]:
tft = TfidfVectorizer(token_pattern=r'(?:\d+|\w+)',
                max_features=500000)
tv_tag = tft.fit_transform(X['user_tags'])

# Kmeans CV

In [6]:
# kmeans_ct = KMeans(n_clusters=10, random_state=0).fit(cv_tag)


In [7]:
# kmeans_tv = KMeans(n_clusters=10, random_state=0).fit(tv_tag)

# Train with linearSVC

In [8]:
train_cv = cv_tag[:1001650]
train_tv = tv_tag[:1001650]
label = train['click'].values

In [9]:
n_fold = 5
sf = StratifiedKFold(n_splits=5,shuffle= True,random_state=233)
train_index_list = []
val_index_list = []
for t,v in sf.split(train_cv,label):
    train_index_list.append(t)
    val_index_list.append(v)

In [18]:
def cross_validation(train_set,label,train_index_list,val_index_list):
    n_fold = len(train_index_list)
    roc_list = []
    log_list = []
    for fold in range(n_fold):
        train_x = train_set[train_index_list[fold]]
        train_y = label[train_index_list[fold]]
        val_x = train_set[val_index_list[fold]]
        val_y = label[val_index_list[fold]]

    #     l_svc = LinearSVC()
    #     lsvc = CalibratedClassifierCV(l_svc) 
    #     lsvc.fit(train_x, train_y)
    #     preds = lsvc.predict_proba(val_x)[:,1]

        lg = LogisticRegression(n_jobs=-1)
        lg.fit(train_x, train_y)
        preds = lg.predict_proba(val_x)[:,1]

        log_score = log_loss(val_y,preds)
        roc_score = roc_auc_score(val_y,preds)
        print('fold {} ends. Logloss is: {}. Roc score is: {}'.format(log_score,log_score,roc_score))

        log_list.append(log_score)
        roc_list.append(roc_score)

    print('{} fold logloss is: {}'.format(n_fold,np.mean(log_list)))
    print('{} fold roc is: {}'.format(n_fold,np.mean(roc_list)))

# validation by count vectorize

In [19]:
cross_validation(train_cv,label,train_index_list,val_index_list)

/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


fold 0.4990524722495905 ends. Logloss is: 0.4990524722495905. Roc score is: 0.5020031475781558
fold 0.498991871747064 ends. Logloss is: 0.498991871747064. Roc score is: 0.4997833290710048
fold 0.4991823056150856 ends. Logloss is: 0.4991823056150856. Roc score is: 0.49763640976498574
fold 0.49905556337606183 ends. Logloss is: 0.49905556337606183. Roc score is: 0.49886218316854825
fold 0.4990624126708926 ends. Logloss is: 0.4990624126708926. Roc score is: 0.4997535497719931
5 fold logloss is: 0.4990689251317389
5 fold roc is: 0.49960772387093755


# validation by tfidf vectorize

In [20]:
cross_validation(train_tv,label,train_index_list,val_index_list)

/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


fold 0.49875487168793053 ends. Logloss is: 0.49875487168793053. Roc score is: 0.5004880903921466
fold 0.4987553948870151 ends. Logloss is: 0.4987553948870151. Roc score is: 0.4996406791257184
fold 0.4989110758010163 ends. Logloss is: 0.4989110758010163. Roc score is: 0.4961165050291107
fold 0.4988452298896855 ends. Logloss is: 0.4988452298896855. Roc score is: 0.49827482874500245
fold 0.49888167538723427 ends. Logloss is: 0.49888167538723427. Roc score is: 0.4987113770185715
5 fold logloss is: 0.4988296495305763
5 fold roc is: 0.49864629606211
